# 추천시스템

## 콘텐츠 기반 필터링(Content based filtering) 실습
- 장르 속성을 이용한 콘텐츠 기반 필터링 만들기

### 1. 데이터 로딩 및 가공

In [1]:
# TMDB 5000영화 데이터 세트
# https://www.kaggle.com/tmdb/tmdb-mivie-metadata

In [2]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [9]:
movies = pd.read_csv('_src/tmdb_5000_movies.csv')
print(movies.shape)
movies.head()

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret...",en,Spectre,A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""name"": ""Danjaq"", ""id"": 10761}, {""name"": ""B24"", ""id"": ...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}, {""iso_3166_1"": ""US"", ""name"": ""United States of ...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}, {""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": ...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""te...",en,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c...",112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""name"": ""Warner Bros."", ""id"": 6194}, {""name"": ""DC E...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fic...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medal...",en,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the myster...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of Ameri

In [10]:
# id, 영화제목title, 장르genres, 평균평점vote_average, 평점 투표수vote_count, 영화인기popularuty, 주요키워드문구keywords, 영화개요설명overview
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [11]:
# 칼럼이 어떤 형태로 돼있는지 확인
pd.set_option('max_colwidth',100) # 보는 옶션 설정
movies_df[['genres','keywords']][:1] 
# 리스트[]안에 {}딕셔너리가 있는 형태의 문자열인것을 확인했다.
# 장르의 명칭은 딕셔너리의 키인 "name"으로 추출가능, keywords도 마찬가지.

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [12]:
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [14]:
# genres컬럼에서 장르명만, keywords컬럼에서 키워드만 리스트로 추출하기
# 리스트 내 여러개의 딕셔너리의 'name'키에 해당하는 값을 찾아 이를 리스트 형태로 변환
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


In [13]:
# 위에 코드를 이중 for문으로 하면?
장르=[]
for x in movies_df['genres'][:1]:
    for y in x:
        장르.append(y['name'])
        print(y['name'])
print(장르)

Action
Adventure
Fantasy
Science Fiction
['Action', 'Adventure', 'Fantasy', 'Science Fiction']


### 2. 장르 콘텐츠 유사도 측정

- 문자열로 반환된 genres 컬럼을 count기반으로 피처 벡터화 변환
- genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교. 이를 위해 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성
- 장르 유사도가 높은 영화중에 평점이 높은 순으로 영화를 추천

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer적용을 위해 공백문자로 word 단위가 구분되는 문자열로 변환.
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",Action Adventure Fantasy Science Fiction
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, ship...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",Adventure Fantasy Action
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi6, british secret service, united kingdom]",A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,Action Adventure Crime
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret identity, burglar, hostage drama, time bomb, gotham...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c...",Action Crime Drama Thriller
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[based on novel, mars, medallion, space travel, princess, alien, steampunk, martian, escape, edg...","John Carter is a war-weary, former military captain who's inexplicably transported to the myster...",Action Adventure Science Fiction


In [23]:
# min_df로 지정한 값보다 작은 경우에는 무시 
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2)) # ngram 단어의 묶음을 1개부터 2개까지 설정(science Fiction같은 2개도 묶음)
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)
#4803개 레코드, 276개 개별단어피처로 구성된 피처벡터행렬 생성

(4803, 276)


In [24]:
# 생성한 행렬에 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2]) # genre_sim객체는 movies_df의 genre_literal칼럼을 피처벡터화한 행렬(genre_mat)데이터의 행별 유사도 정보를 갖고있다.
# 장르기준으로 콘텐츠기반필터링을 하려면 genre_sim객체 이용해서 가장 장르 유사도가 높은 순으로 레코드 추출해야함.

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [27]:
# 유사도가 가장 높은 순으로 정렬
genre_sim_sorted_index = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_index[:1])
# 0번 레코드는 3494번 레코드가 가장 유사도가 높고, 그다음이 813레코드, 가장 유사도가 낮은 레코드는 2401번이다.

[[   0 3494  813 ... 3038 3037 2401]]


### 3. 장르 콘텐츠 필터링을 이용한 영화 추천

In [39]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10): # title_name : 추천기준 영화이름, top_n : 상위 10개만 추천
    # movies_df 'title'컬럼에 입력된 title_name값인 df출력
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DF의 index 객체를 ndarray로 반환
    title_index = title_movie.index.values
    # genre_sim_sorted_index객체에서 유사도 순으로 top_n개 index추출
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # top_n index출력. (2차원->1차원으로)
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [40]:
movies_df[movies_df['title'] == 'The Godfather']

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
3337,238,The Godfather,"[Drama, Crime]",8.4,5893,143.659698,"[italy, love at first sight, loss of father, patriarch, organized crime, mafia, lawyer, italian ...","Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American Corleone crime fa...",Drama Crime


In [41]:
# 만든 함수를 이용해서 영화'대부'와 장르별로 유사한 상위 10개 영화를 추천
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_index, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]
# kids등 대부를 좋아하는 사용자에게 추천하기 맞지 않아보이고, 평점이 0인 영화도 있으므로 개선이 필요해보임.

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [42]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10] # False 내림차순
# 평균평점순으로 정렬했더니 유사성이 낮아보이는 still Upper Lips이 더 높은 평점으로 추천되었다. 투표수를 보니 평가횟수가 1회로 매우적다.
# 왜곡된 평점데이터를 회피하기위해 유명한 영화 평점 사이트인 IMDB에서 가중 평점 방식을 사용.

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- 가중 평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C
    - v : 개별 영화에 평점을 투효한 횟수
    - m : 평점을 부여하기 위한 최소 투표 횟수
    - R : 개별 영화에 대한 평균 평점
    - C : 전체 영화에 대한 평균 평점

In [44]:
# v = 'vote_count'
# R = 'vote_average'
# C = movies_df['vote_average'].mean()
# m = 값이 높으면 평점 투표 횟수가 많은 영화에 더 많은 가중 평점을 부여. 상위 60%에 해당하는 횟수를 기준(pandas 함수명은 quantile( ), numpy 함수명은 percentile( ))

C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:', round(C,3), 'm:', round(m,3))

C: 6.092 m: 370.2


In [57]:
# 기존 평점을 새로운 가중 평점으로 변경하는 함수 생성하고 새로운 평점 정보는 'weighted_vote'컬럼을 생성해서 출력
percentile = 0.6
m = movies['vote_count'].quantile(percentile)
C = movies['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v/(v+m)) * R) + ((m/(v+m)) * C)

movies['weighted_vote'] = movies.apply(weighted_vote_average, axis=1)

In [58]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [59]:
# 새로운 평점 기준에 따라서 영화를 추천해보기
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # 장르 유사성이 높은 영화 top_n의 2배수만큼 후보선정
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    
    similar_indexes = similar_indexes[similar_indexes != title_index] # 기존 영화 인덱스 제외
    
    # weighted_vote 값이 높은순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_index, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


## 아이템 기반 최근접 이웃 협업 필터링 실습

### 1. 데이터 가공 및 변환

In [ ]:
# MovieLens 데이터 셋 : 십만개의 평점(ratine)정보
# https://grouplens.org/datasets/movielens/latest

In [69]:
import pandas as pd
import numpy as np

In [76]:
movies = pd.read_csv('_src/ml-latest-small/movies.csv')
ratings = pd.read_csv('_src/ml-latest-small/ratings.csv')
print(movies.shape)
print(ratings.shape)

(9742, 3)
(100836, 4)


In [77]:
# 영화에 대한 정보 (제목,장르)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [78]:
# 사용자별로 영화에 대한 평점을 매긴 데이터 셋
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [79]:
# 인덱스를 userId로, 컬럼을 movieId로 변경
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
ratings_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
# movieId 컬럼을 1,2,3,4가 아니라 영화이름으로 모두 변경하기
# movies df의 'title' 컬럼을 얻기위해 movies와 조인
rating_movies = pd.merge(ratings, movies, on='movieId')

# title컬럼으로 피벗 수행
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

# NaN값을 모두 0으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. 영화간 유사도 산출
- 코사인 유사도 cosine_similarity()를 기반으로 측정

In [83]:
# cosine_similarity()는 행을 기준으로 다른 행과의 유사도를 측정
# 현재 데이터로는 사용자간 유사도가 측정됨 -> 행,열 바꿔야함
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity로 반환된 넘파이 행렬을 영화명을 매핑해 df로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head()

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Salem's Lot (2004),0.0,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Til There Was You (1997),0.0,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [87]:
# "Godfather, The (1972)"와 유사도가 높은 상위 6개 영화추출
item_sim_df["Godfather, The (1972)"].sort_values(ascending=False)[:6]

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

In [90]:
# 인셉션과 유사도가 높은 영화 추출
item_sim_df["Inception (2010)"].sort_values(ascending=False)[1:6]

title
Dark Knight, The (2008)          0.727263
Inglourious Basterds (2009)      0.646103
Shutter Island (2010)            0.617736
Dark Knight Rises, The (2012)    0.617504
Fight Club (1999)                0.615417
Name: Inception (2010), dtype: float64

### 3. 아이템 기반 최근접 이웃 협업 필터링으로 개인화된 영화 추천
- 개인화된 영화 추천은 개인이 아직 관람하지 않은 영화를 추천한다는 것.
- 아직 관람하지 않은 영화에 대해서 아이템 유사도와 기존에 관람한 영화의 평점 데이터를 기반으로 예측평점을 계산
- 높은 예측평점을 가진 영화를 추천

In [91]:
# 예측평점 구하는 식을 함수로 구현
def predict_rating(ratings_arr, item_sim_arr ):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [93]:
# 만든 함수를 이용해 개인화된 예측 평점 구하기
# ratings_matrix와 item_sim_df를 넘파이 행렬로 변환해 인자로 입력
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred

array([[0.07034471, 0.5778545 , 0.32169559, ..., 0.13602448, 0.29295452,
        0.72034722],
       [0.01826008, 0.04274424, 0.01886104, ..., 0.02452792, 0.01756305,
        0.        ],
       [0.01188449, 0.03027871, 0.06443729, ..., 0.00922874, 0.01041982,
        0.08450144],
       ...,
       [0.32443466, 1.02254119, 0.5984666 , ..., 0.53858621, 0.52763888,
        0.69887065],
       [0.00483488, 0.05359271, 0.02625119, ..., 0.0131077 , 0.01832826,
        0.03337679],
       [3.62830323, 1.51791811, 0.83366781, ..., 1.81609065, 0.56507537,
        0.57465402]])

In [95]:
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
ratings_pred_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501
4,0.049145,0.277628,0.160448,0.206892,0.309632,0.042337,0.130048,0.116442,0.099785,0.097432,...,0.051269,0.076051,0.055563,0.054137,0.008343,0.159242,0.100941,0.062253,0.146054,0.231187
5,0.007278,0.066951,0.041879,0.013880,0.024842,0.018240,0.026405,0.018673,0.021591,0.018841,...,0.009689,0.022246,0.013360,0.012378,0.000000,0.025839,0.023712,0.018012,0.028133,0.052315


- 기존에 영화를 관람하지 않아서 실제평점이 0인 값이 예측에서는 값이 부여됨
- 예측결과가 원래의 실제 평점과 얼마나 차이가 있는지 확인
- 예측평가지표는 MSE적용
- 실제와 예측 평점의 차이는 기존 평점데이터에 대해서만 오차 정도를 측정

In [96]:
from sklearn.metrics import mean_squared_error
 
# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE를 구함.
def get_mse(pred, actual):
    # 평점이 있는 실제 영화만 추출 
    pred = pred[actual.nonzero()].flatten() # nonzero : 요소들 중 0이 아닌 값들의 index 들을 반환해 주는 함수
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

아이템 기반 모든 인접 이웃 MSE:  9.895354759094706


In [98]:
# 많은 영화의 유사도 벡터를 이용하다보니 상대적으로 평점예측이 떨어짐
# 특정영화와 가장 비슷한 유사도를 가진 영화에 대해서만 유사도 벡터를 적용하는 함수로 변경하기
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)
 
    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행.
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))       
    return pred

In [99]:
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))
 
# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
# MSE가 3.69로 기존보다 많이 향상되었다.

아이템 기반 인접 TOP-20 이웃 MSE:  3.695009387428144


In [101]:
# userId = 9인 사용자에 대해 영화 추천
# 9번 사용자가 평점을 준 영화를 평점이 높은 순으로 나열
user_rating_id = ratings_matrix.loc[9, :]
user_rating_id[user_rating_id > 0].sort_values(ascending=False)[:10]

title
Adaptation (2002)                                                                 5.0
Austin Powers in Goldmember (2002)                                                5.0
Lord of the Rings: The Fellowship of the Ring, The (2001)                         5.0
Lord of the Rings: The Two Towers, The (2002)                                     5.0
Producers, The (1968)                                                             5.0
Citizen Kane (1941)                                                               5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Back to the Future (1985)                                                         5.0
Glengarry Glen Ross (1992)                                                        4.0
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)                                     4.0
Name: 9, dtype: float64

In [102]:
# 9번 사용자에게 아이템기반 협업 필터링을 통해 영화를 추천
# 평점을 주지 않은 영화(안본영화)를 리스트 객체로 반환하는 함수 생성

def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함.
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임.
    user_rating = ratings_matrix.loc[userId,:]
     
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()
     
    # 모든 영화명을 list 객체로 만듬.
    movies_list = ratings_matrix.columns.tolist()
     
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함.
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]
     
    return unseen_list

In [108]:
# 사용자가 안본 영화 정보와 예측평점데이터 세트를 이용해서 최종적으로 영화를 추천하는 함수
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함.
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [110]:
# 사용자가 관람하지 않는 영화명 추출  
unseen_list = get_unseen_movies(ratings_matrix, 9)
 
# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)
 
# 평점 데이터를 DataFrame으로 생성.
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591


## 행렬분해를 이용한 잠재요인 협업 필터링 실습

### 1. SGD 행렬분해에 대한 간단한 계산 예제

#### SGD 절차
1. P와 Q행렬을 임의의 값을 가진 행렬로 초기화한다.
2. P와 Q전치행렬을 곱해 예측R행렬을 계산하고, 실제R행렬과의 차이를 계산한다.
3. 차이를 최소화할 수 있도록 P와 Q행렬의 값을 적절한 값으로 각각 업데이트한다.
4. 특정 임계치 아래로 수렴할때까지 2,3번 작업을 반복하면서 P와 Q행렬을 업데이트해 근사화한다.

In [1]:
# SGD 행렬분해에 대한 간단한 계산 예제
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3 설정.
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])
num_users, num_items = R.shape # R.shape (4,5)
K=3
print('실제 행렬:\n', R)

# P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 random한 값으로 입력합니다.
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

실제 행렬:
 [[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]


In [2]:
# 실제 R과 예측 R 행렬의 오차를 구하는 함수 생성
from sklearn.metrics import mean_squared_error
def get_rmse(R,P,Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 null이 아닌 값의 위치인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
       
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
     
    return rmse

- 이제 SGD기반으로 행렬 분해를 수행.
- 먼저 R에서 null값을 제외한 데이터의 행렬 인덱스 추출

In [3]:
print(num_users, num_items)

4 5


In [4]:
# R > 0이면 행 위치(i), 열 위치(j), 값을 non_zeros리스트에 저장.
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
non_zeros

[(0, 0, 4.0),
 (0, 3, 2.0),
 (1, 1, 5.0),
 (1, 3, 3.0),
 (1, 4, 1.0),
 (2, 2, 3.0),
 (2, 3, 4.0),
 (2, 4, 4.0),
 (3, 0, 5.0),
 (3, 1, 2.0),
 (3, 2, 1.0),
 (3, 3, 2.0)]

- steps : SGD를 반복해서 업데이트할 횟수
- learning_rate : SGD의 학습률
- r_lambda : L2 Regularizaiton 계수

In [5]:
steps = 1000
learning_rate = 0.01
r_lambda = 0.01
 
# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트.
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
 
    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  3.2388050277987723
### iteration step :  50  rmse :  0.4876723101369648
### iteration step :  100  rmse :  0.1564340384819247
### iteration step :  150  rmse :  0.07455141311978046
### iteration step :  200  rmse :  0.04325226798579314
### iteration step :  250  rmse :  0.029248328780878973
### iteration step :  300  rmse :  0.022621116143829466
### iteration step :  350  rmse :  0.019493636196525135
### iteration step :  400  rmse :  0.018022719092132704
### iteration step :  450  rmse :  0.01731968595344266
### iteration step :  500  rmse :  0.016973657887570753
### iteration step :  550  rmse :  0.016796804595895633
### iteration step :  600  rmse :  0.01670132290188466
### iteration step :  650  rmse :  0.01664473691247669
### iteration step :  700  rmse :  0.016605910068210026
### iteration step :  750  rmse :  0.016574200475705
### iteration step :  800  rmse :  0.01654431582921597
### iteration step :  850  rmse :  0.01651375177473524
### iterati

In [6]:
# 분해된 P와 Q 함수를 P*Q.T로 예측행렬을 만들어서, 실제 행렬과 비교
pred_matrix = np.dot(P,Q.T)
print('예측 행렬:\n', np.round(pred_matrix,3))
print('실제 행렬:\n', R)
# 비교해본 결과 null이 아닌 값은 큰 차이가 나지 않으며, null인 값은 새로운 예측값으로 채워졌다.

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]
실제 행렬:
 [[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]


### 2. 행렬분해를 이용한 개인화된 영화 추천 시스템 개발

- 확률적 경사하강법을 이용한 get_rmse()함수를 이용하여 행렬분해 로직을 새로운 함수로 정리
    - steps : SGD를 반복해서 업데이트할 횟수
    - learning_rate : SGD의 학습률
    - r_lambda : L2 Regularizaiton 계수

In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

In [15]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape # (4,5)
    
    # P와 Q매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력
    np.random.seed(1)    
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
    
    prev_rmse = 10000
    break_count = 0
    
    # R > 0이면 행 위치(i), 열 위치(j), 값을 non_zeros리스트에 저장.
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
    
    # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트.
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
    return P,Q

In [16]:
if __name__ == "__main__":
    import pandas as pd
    import numpy as np

    movies = pd.read_csv('_src/ml-latest-small/movies.csv')
    ratings = pd.read_csv('_src/ml-latest-small/ratings.csv')
    ratings = ratings[['userId', 'movieId', 'rating']]
    ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

    # title 컬럼을 얻기 이해 movies 와 조인 수행
    rating_movies = pd.merge(ratings, movies, on='movieId')

    # columns='title' 로 title 컬럼으로 pivot 수행
    ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

    # 새로만든 함수를 이용해서 행렬분해 수행
    P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, 
                                learning_rate=0.01, r_lambda = 0.01)
    pred_matrix = np.dot(P,Q.T)

### iteration step :  0  rmse :  2.9023619751336867
### iteration step :  10  rmse :  0.7335768591017927
### iteration step :  20  rmse :  0.5115539026853442
### iteration step :  30  rmse :  0.37261628282537446
### iteration step :  40  rmse :  0.2960818299181014
### iteration step :  50  rmse :  0.2520353192341642
### iteration step :  60  rmse :  0.22487503275269854
### iteration step :  70  rmse :  0.20685455302331537
### iteration step :  80  rmse :  0.19413418783028685
### iteration step :  90  rmse :  0.18470082002720403
### iteration step :  100  rmse :  0.17742927527209104
### iteration step :  110  rmse :  0.1716522696470749
### iteration step :  120  rmse :  0.1669518194687172
### iteration step :  130  rmse :  0.16305292191997542
### iteration step :  140  rmse :  0.15976691929679643
### iteration step :  150  rmse :  0.1569598699945732
### iteration step :  160  rmse :  0.15453398186715428
### iteration step :  170  rmse :  0.15241618551077643
### iteration step :  180  rm

In [17]:
# 쉽게 이해하기 위해 반환된 예측 사용자-아이템 평점 행렬을 영화 타이틀을 컬럼명으로 가지는 DF로 변경
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=ratings_matrix.index,
                                   columns=ratings_matrix.columns)
 
ratings_pred_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941
4,2.628629,3.035550,2.575746,3.706912,3.430636,0.706441,3.330280,1.978826,4.560368,2.775710,...,1.046116,2.912178,2.479592,2.231915,1.888629,2.211364,0.645603,1.585734,3.542892,0.591540
5,2.116148,3.084761,2.747679,3.783490,3.946990,0.883259,1.958953,1.757317,2.054312,2.775258,...,0.956159,3.893975,2.717024,2.002443,2.053337,3.983639,2.099626,1.423718,2.490428,0.531403


In [23]:
# 사용자가 관람하지않은 영화만 추출해서 추천하기 위해 get_unseen_movies함수,recomm_movie_by_userid함수 다시 사용
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함.
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임.
    user_rating = ratings_matrix.loc[userId,:]
     
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()
     
    # 모든 영화명을 list 객체로 만듬.
    movies_list = ratings_matrix.columns.tolist()
     
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함.
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]
     
    return unseen_list


def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함.
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [24]:
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 인접 이웃 협업 필터링으로 관람하지 않은 영화 상위 10개 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 DataFrame으로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index,
                            columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


## Surprise

- Surprise 는 사용자 아이디, 아이템 아이디, 평점 데이터가 로우 레벨로 된 데이터 세트만 적용할 수 있다. 
- 그래서 데이터의 첫번째 컬럼을 사용자 아이디, 두번째 컬럼을 아이템 아이디, 세번째 컬럼을 평점으로 가정해 데이터를 로딩하고 네번째 컬럼부터는 로딩을 수행하지 않는다. 
    - 예로, user_id, item_id, rating, time_stamp 필드로 구분된 데이터라면 앞 3개 필드만 로딩하고 이후 time_stamp 필드는 로딩에서 제외
    - 따라서 반드시 데이터 세트의 컬럼 순서가 사용자 아이디, 아이템 아이디, 평점 순으로 돼 있어야 한다.

https://big-dream-world.tistory.com/70?category=470634

### 1. Surprise를 이용한 추천 시스템 구축

- http://sruprise.readthedocs.io/en/stable/ 에 정리되어있다
- 간단한 예제 : 추천데이터를 학습용과 테스트용 데이터세트로 분리한 뒤 SVD 행렬분해를 통해 잠재요인협업필터링 수행하기

In [4]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [5]:
data = Dataset.load_builtin('ml-100k')
# 수행시마다 동일하게 데이터를 분할하기 위해 random_state값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

- surprise는 자체적으로 로우레벨의 데이터를 칼럼 레벨의 데이터로 변경하므로 원본인 로우 레벨의 사용자-아이템 데이터를 데이터 세트로 적용해야한다.

In [6]:
# SVD로 잠재 요인 협업 필터링 수행
algo=SVD() # 알고리즘 객체 생성
algo.fit(trainset)

- surprise에서 추천을 예측하는 메서드는 test()와 predict()이다.
- test()는 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드
- predict()는 개별 사용자와 영화에 대한 추천 평점을 반환하는 메서드

In [7]:
# 테스트데이터 세트에 대해 추천 수행
predictions = algo.test(testset)
print('prediction type:', type(predictions), 'size:',len(predictions))
print('prediction 결과의 최초 5개 추출:')
predictions[:5]
# 개별 사용자 아이디(uid), 영화(아이템)아이디(iid), 실제 평점(r_ui)정보에 기반해 surprise의 추천 예측 평점(est)데이터를 튜플 형태로 갖고있다.
# was_impossible이 True면 예측값을 생성할 수 없는 데이터라는 의미이다.

prediction type: <class 'list'> size: 25000
prediction 결과의 최초 5개 추출:


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.37191922849625, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.7922231452759703, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.9819408831879834, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.723998754156991, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.560114580554585, details={'was_impossible': False})]

In [10]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.37191922849625),
 ('882', '291', 3.7922231452759703),
 ('535', '507', 3.9819408831879834)]

In [11]:
# predict로 추천예측해보기
# 사용자 아이디, 아이템 아이디를 문자열로 입력 -> 추천 예측 평점을 est로 반환
uid=str(196)
iid=str(302)
pred=algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.25   {'was_impossible': False}


In [12]:
accuracy.rmse(predictions)

RMSE: 0.9485


0.9484676724192952

### 02. OS파일 데이터를 surprise데이터 세트로 로딩

In [14]:
import pandas as pd
ratings=pd.read_csv('_src/ml-latest-small/ratings.csv')

# ratings_noh.csv파일로 언로드시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('_src/ratings_noh.csv', index=False, header=False)

- Reader 클래스의 주요 생성 파라미터
    - line_format(string) : 칼럼을 순서대로 나열. 입력된 문자열을 공백으로 분리해 칼럼으로 인식
    - sep(char) : 칼럼을 분리하는 분리자(문자), 디폴트는 '\t'
    - rating_scale(tuple,optional) : 평점 값의 최소~ 최대 설정. 디폴트는(1,5)

In [19]:
from surprise import Reader

# 컬럼의 분리문자는 콤마, 평점 단위는 0.5, 최대평점 5
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('_src/ratings_noh.csv', reader=reader)

In [20]:
# SVD 행렬분해기법을 이용해 추천 예측 수행
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정
# 잠재요인 크기 K값을 나타내는 파라미터인 n_facctors 50으로 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 테스트 데이터로 평점 예측 후 rmse 평가
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### 03. DataFrame에서 surprsie 데이터 세트로 로딩
- 주의할 점은 사용자 아이디, 아이템 아이디, 평점 컬럼 순서를 지켜야 한다.

In [28]:
import pandas as pd
from surprise import Reader, Dataset
 
ratings = pd.read_csv('_src/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
 
# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [27]:
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### 04. 교차검증과 하이퍼 파라미터 튜닝

- Surprise 는 교차 검증과 하이퍼 파라미터 튜닝을 위해 사이킷런과 유사한 cross_validate() 와 GridSearchCV 클래스를 제공한다.
- cross_validate()는 인자로 알고리즘 객체, 데이터, 성능 평가 방법, 폴드 데이터 세트 개수(cv)를 입력받아, 출력결과와 같이 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 함께 보여준다.

In [32]:
from surprise.model_selection import cross_validate
 
# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩
ratings = pd.read_csv('_src/ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
 
algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
# 폴드별 성능평가수치와 전체 폴드의 평균 성능평가수치를 출력

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8788  0.8692  0.8694  0.8787  0.8707  0.8734  0.0044  
MAE (testset)     0.6746  0.6680  0.6691  0.6755  0.6678  0.6710  0.0034  
Fit time          8.82    7.58    9.84    9.06    9.05    8.87    0.73    
Test time         0.18    0.19    0.20    0.58    0.29    0.29    0.15    


{'test_rmse': array([0.87882661, 0.86916342, 0.86941893, 0.87869138, 0.87073822]),
 'test_mae': array([0.67457549, 0.6679954 , 0.66905003, 0.67551973, 0.66781015]),
 'fit_time': (8.820579528808594,
  7.5778021812438965,
  9.838555335998535,
  9.056925773620605,
  9.053822994232178),
 'test_time': (0.1819441318511963,
  0.19405698776245117,
  0.20430326461791992,
  0.5783681869506836,
  0.29001450538635254)}

- Surprise 는 GridSearchCV를 통해 교차 검증을 통한 하이퍼 파라미터 최적화를 수행할 수 있다. 
- 하이퍼 파라미터 최적화는 알고리즘 유형에 따라 다를 수 있지만, SVD의 경우 주로 SGD(점진적 하강 방식)의 반복 횟수를 지정하는 n_epochs와 SVD의 잠재 요인 K의 크기를 지정하는 n_factors를 튜닝한다. 

In [33]:
from surprise.model_selection import GridSearchCV
 
# 최적화할 파라미터들을 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }
 
# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)
 
# CV가 3일때 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
# n_epochs:20, n_factors:50일때 3개 폴드의 검증 데이터 세트에서 최적 RMSE가 약 0.8769로 도출되었다.

0.8768328890717677
{'n_epochs': 20, 'n_factors': 50}
